In [19]:
import tensorflow

In [20]:
# https://becominghuman.ai/constructing-a-cnn-network-for-dogs-and-cats-dataset-4c76b475e435
import cv2
import numpy as np
import os  
from random import shuffle 
from tqdm import tqdm

In [21]:
# images dir 
IMG_SIZE = 100
images_dir = 'dogs-vs-cats/train'
# function that looks at file name to check whether it's a dog or cat in training data
def label_img(img):
    word_label = img.split('.')[-3]
    # conversion to one-hot array [cat,dog]
    #  [much cat, no dog]
    if word_label == 'cat': return 1
    #  [no cat, very doggo]
    elif word_label == 'dog': return 0
# creates training data (converts images to grayscale)
def process_data():
    my_data = []
    my_result = []
    # loop through images in training data directory
    for img in tqdm(os.listdir(images_dir)):
        # set the label for the pixel data - either 1 (cat) or 0 (dog)
        label = label_img(img)
        # get training data image
        path = os.path.join(images_dir,img)
        img = cv2.imread(path)
        #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gray = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        #gray = gray.flatten()
        my_data.append(np.array(gray))
        my_result.append(label)
    np.save('my_data.npy', my_data)
    np.save('my_result.npy', my_result)
    return my_data, my_result

In [22]:
X, y = process_data()

100%|██████████| 25000/25000 [00:33<00:00, 744.02it/s]


In [23]:
X = np.array(X)
X = X/255.

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=(2/3), random_state=0)

In [26]:
X_train.shape
y_train[:5]

[1, 0, 0, 0, 0]

In [27]:
from tensorflow.keras import layers, models

In [11]:
X_test.shape

(8334, 100, 100, 3)

In [11]:
ann = models.Sequential([
        layers.Flatten(input_shape=(100,100,3)),
        layers.Dense(3000, activation='relu'),
        layers.Dense(500, activation='relu'),
        layers.Dense(125, activation='relu'),
        layers.Dense(25, activation='relu'),
        layers.Dense(10, activation='softmax')    
    ])

ann.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [66]:
ann.fit(X_train, np.array(y_train), epochs=5)

Epoch 1/5
521/521 [==============================] - 48s 91ms/step - loss: 0.7898 - accuracy: 0.5042
Epoch 2/5
521/521 [==============================] - 49s 95ms/step - loss: 0.6788 - accuracy: 0.5847
Epoch 3/5
521/521 [==============================] - 47s 91ms/step - loss: 0.6642 - accuracy: 0.6021
Epoch 4/5
521/521 [==============================] - 48s 91ms/step - loss: 0.6507 - accuracy: 0.6156
Epoch 5/5
521/521 [==============================] - 48s 91ms/step - loss: 0.6487 - accuracy: 0.6293


In [67]:
results = ann.evaluate(X_test, np.array(y_test), batch_size=521)

16/16 [==============================] - 13s 784ms/step - loss: 0.6714 - accuracy: 0.5769


In [28]:
cnn = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(100, 100, 3)),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])
cnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [29]:
cnn.fit(X_train, np.array(y_train), epochs=3)

Epoch 1/3
521/521 [==============================] - 28s 53ms/step - loss: 0.7101 - accuracy: 0.6028
Epoch 2/3
521/521 [==============================] - 27s 52ms/step - loss: 0.4990 - accuracy: 0.7618
Epoch 3/3
521/521 [==============================] - 27s 52ms/step - loss: 0.4065 - accuracy: 0.8162


In [ ]:
cnn_results = cnn.evaluate(X_test, np.array(y_test), batch_size=521, verbose=0)